In [114]:
import requests
import os
from langchain.docstore.document import Document
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI 
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma
import re

from dotenv import load_dotenv
load_dotenv()

base_url = os.environ["SERVICENOW_BASEURL"]
user = os.environ["SERVICENOW_USERNAME"]
pwd = os.environ["SERVICENOW_PASSWORD"]
headers = {"Content-Type":"application/json","Accept":"application/json"}


def get_specific_catalog_item(sys_id):

    url = f"{base_url}/sn_sc/servicecatalog/items/{sys_id}"
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print("Status:", response.status_code, "Headers:", response.headers, "Error Response:",response.json())
        exit()
    data = response.json()
    print(data)
    return data




def get_table_response(reference):
    url = f"{base_url}/now/table/{reference}?sysparm_limit=1"
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print("Status:", response.status_code, "Headers:", response.headers, "Error Response:",response.json())
        exit()
    data = response.json()
    return data


def get_table_sysid(reference, table_field, value_for_query):
    if("@" in value_for_query):
        value_for_query = value_for_query.replace("@","%40")
    sysparm_query = f"{table_field}%3D{value_for_query}" 

    url = f"{base_url}/now/table/{reference}?sysparm_query={sysparm_query}&sysparm_limit=1"
    # print(url)
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print("Status:", response.status_code, "Headers:", response.headers, "Error Response:",response.json())
        exit()

    data = response.json()
    return data

def get_table_basedon_ref_qualifier(reference, sys_param_query):
    url = f'{base_url}/now/table/{reference}?sysparm_query={sys_param_query}'
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
        exit()
    data = response.json()
    print(data)


def get_catalog_item_variables(sys_id):
    url = f"{base_url}/sn_sc/servicecatalog/items/{sys_id}/variables"
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print("Status:", response.status_code, "Headers:", response.headers, "Error Response:",response.json())
    print(response)
    data = response.json()
    return data

def get_table_values(reference):
    url = f"{base_url}/now/table/{reference}"
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print("Status:", response.status_code, "Headers:", response.headers, "Error Response:",response.json())
        exit()
    data = response.json()
    return data


In [115]:
from langchain.schema import HumanMessage
import json
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI 

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

In [116]:
catalog_item_description = """

Catalog Item Name: Software Services

Description: This catalog item allows users to request various software services, including OS upgrades, software installation, and uninstallation. Please select the appropriate business function/request type to display the delivery time. This service is available on desktop, and you can add multiple software items if required.

Variables (Fields):

1. Requested For: This is a mandatory field that specifies the user for whom the software service is requested. The default value is set to "System Administrator."

2. Contact Number: This is a mandatory field where you need to enter your contact number to facilitate communication.

3. Location: This field is not mandatory and allows you to specify the location related to the request. The location will be dynamically fetched based on the "Requested For" field.

4. Project Name: This field is not mandatory and allows you to specify the project associated with the request. The project name will be dynamically fetched based on the "Requested For" field.

5. Business Service: This is a mandatory field where you need to select the type of business service required. The available choices are:
   - OS Upgrade
   - Software Installation
   - Software Uninstallation
   
   UI Policy: 
   - If the business service is "Software Installation," the following fields become mandatory and visible:
     - Request Type
     - Software Name
     - Duration
     - Add Software (visible but not mandatory)

6. Software Name: This field is not mandatory and allows you to specify the software required. The available choices include:
   - PowerShell
   - Swift
   - COBOL
   - Minitab
   - Verilog
   - Flutter SDK
   - Xcode
   - Overleaf
   - Microsoft PowerPoint
   - Node.js SDK
   - Erlang
   - Power BI
   - InVision
   - Apache Cordova SDK
   - Common Lisp
   - RStudio
   - JavaScript
   - Google Chrome
   - Dart
   - AutoCAD
   - QlikView
   - Scala
   - Julia
   - Perl
   - R
   - Prolog
   - Box SDK
   - HTML
   - Celtx
   - Microsoft Word
   - Scikit-learn
   - Haskell
   - .NET SDK
   - SolidWorks
   - IntelliJ IDEA
   - Slack
   - React Native SDK
   - PyCharm
   - Jenkins
   - CSS
   - Go (Golang)
   - PayPal SDK
   - QuickBooks
   - iOS SDK
   - Facebook SDK
   - TensorFlow SDK
   - SQLite
   - Docker
   - Apache Spark
   - Groovy
   - Grammarly
   - Azure SDK
   - Notepad++
   - Java Development Kit (JDK)
   - TypeScript
   - Sketch
   - Dropbox SDK
   - Spring Framework
   - Google Maps SDK
   - Elixir
   - NVivo
   - Sublime Text
   - Adobe Photoshop
   - VHDL
   - MongoDB
   - Adobe After Effects
   - Assembly Language
   - Mixpanel SDK
   - Unreal Engine SDK
   - PHP
   - Stata
   - CorelDRAW
   - GitLab
   - D
   - Sass
   - Bitbucket
   - AWS SDK
   - Visual Studio Code
   - Xero
   - SAP
   - RPG Maker
   - MATLAB
   - OpenCV SDK
   - Affinity Photo
   - Crystal
   - Firebase SDK
   - Rust
   - Java
   - Opera
   - Zotero
   - 3ds Max
   - C++
   - Kubernetes
   - Figma
   - Asana
   - Shell Scripting (Bash)
   - Google Cloud SDK
   - LaTeX
   - Trello
   - EndNote
   - Slack SDK
   - Android Studio
   - C#
   - SketchUp
   - PostgreSQL
   - Pascal
   - ARKit
   - Fortran
   - Microsoft Teams
   - SAS
   - SAP SDK
   - Amplitude SDK
   - Salesforce SDK
   - VB.NET
   - Telegram Bot SDK
   - Lua
   - Less
   - Zoom
   - Nim
   - Android SDK
   - Adobe Premiere Pro
   - Vagrant
   - Unity
   - OneSignal SDK
   - Eclipse
   - Qt SDK
   - Hadoop
   - Kotlin SDK
   - Mendeley
   - Scrivener
   - NoSQL
   - Keras
   - C
   - Mozilla Firefox
   - Procreate
   - SPSS
   - Tcl
   - Adobe Illustrator
   - Oracle Database
   - Scheme
   - ARCore
   - Stripe SDK
   - Tableau
   - Sage 50
   - TensorFlow
   - Objective-C
   - Ansible
   - Adobe Acrobat Reader
   - GitHub
   - Cassandra
   - Swift SDK
   - Blender
   - Unreal Engine
   - Google Meet
   - Microsoft Excel
   - Zoom SDK
   - Affinity Designer
   - Python
   - Ada
   - Terraform
   - SQL
   - Xamarin SDK
   - Realm SDK
   - NetBeans
   - GTK+ SDK
   - Electron SDK
   - Salesforce
   - Unity SDK
   - Jupyter Notebook
   - Redis
   - Twilio SDK
   - Agora SDK
   - Segment SDK
   - Twitter SDK
   - PyTorch
   - Intercom SDK
   - F#
   - Safari
   - Atom
   - Sentry SDK
   - Windows SDK
   - Jira
   - Skype
   - GameMaker Studio
   - Discord SDK
   - Microsoft SQL Server
   - Final Draft
   - Clojure
   - Cocoa Touch
   - Mathematica
   - Canva
   - FreshBooks
   - Ruby
   - Kotlin
   - Maya
   - MySQL
   - Microsoft Edge
   - New Relic SDK

7. Add Software: This is a check box that allows you to add more software items if required. When checked, additional fields for specifying software details will become visible and mandatory.

   UI Policy:
   - If the checkbox is selected, the following fields become mandatory and visible:
     - Software Name1
     - Duration1
     - Request Type1
     - Add Software1 (visible but not mandatory)

8. Request Type: This field is not mandatory and allows you to specify the type of request for the selected software.

9. Duration: This field is not mandatory and allows you to specify the duration for which the software service is required. The available choices are:
   - None
   - 30 days
   - 60 days
   - 90 days
   - 180 days
   - 360 days

10. Select your Host Name: This is a mandatory lookup select box where you need to select your host name. The choices are dynamically fetched based on the "Requested For" field. The available choices include:
    - HostName1
    - HostName2
    - HostName3

11. Additional Software Fields: Similar to the first software field, additional fields are provided if the "Add Software" checkbox is selected, allowing users to specify more software items, request types, and durations. The same choices and conditions apply to these fields.

"""

In [117]:
def fetchVariables(data, variables_name):

    template = """{data}
    
    from the given data Return only the variables name as a key and variable description as a value in JSON.
    The description must describe contains choices and ui_policy too as a single output
    The variablename should be one of the name from the given List. {variables_name}
    And no premable or explaination.

    Example:
    variablename : variabledescription

    """
    
    prompt_template = PromptTemplate(template=template, input_variables=["data", variables_name])
    chain = prompt_template | model | JsonOutputParser()
    response = chain.invoke({"data": data, "variables_name": variables_name})
    print(response)
    return response

In [118]:
def create_custom_function(variable_info, custom_functions, variables_description):
    inner_json = {}
    inner_json["type"] = "string"
    if(variable_info["name"] in variables_description):
        inner_json["description"] = variables_description[variable_info["name"]]
        custom_functions[0]["parameters"]["properties"][variable_info["name"]] = inner_json
    return custom_functions


def function_calling_catVar(catalog_variables, catalog_item_description, variables_name):
    custom_functions = [
        {
            "name": "extract_catalog_variables",
            "description": "Get the values from the body of the user query",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
        ]
    variables_description = fetchVariables(catalog_item_description, variables_name)


    for variable_info in catalog_variables:
        if "children" in variable_info:
            for child_var_info in variable_info["children"]:
                # print(child_var_info)
                custom_functions = create_custom_function(child_var_info, custom_functions, variables_description)
        else:
            custom_functions = create_custom_function(variable_info, custom_functions, variables_description)
    return custom_functions


In [119]:
def get_all_variables_List(catalog_item_variable):
    def extract_variables(variables):
        variable_List = []
        for variable_info in variables:
            if variable_info["type"] == 24 or variable_info['type'] == 11:
                continue
            if "children" in variable_info:
                variable_List.extend(extract_variables(variable_info["children"]))
            else:
                variable_List.append(variable_info)
        return variable_List
    return extract_variables(catalog_item_variable)


def set_reference_value(variable):
    
    if(variable["type"] == 8 and variable["dynamic_value_field"] == ""):
        # if(variable['ref_qualifier'] != ''):
        #     ref_qualifier = variable['ref_qualifier']
        #     if(not ref_qualifier.startswith('javascript:new')):
        #         reference = variable["reference"]
        #         table_content = get_table_basedon_ref_qualifier(reference, ref_qualifier)
        #     else:
        #         pass
        # else:
        #     reference = variable["reference"]
        #     table_content = get_table_values(reference=reference)
        
        reference = variable["reference"]
        table_content = get_table_values(reference=reference)

        names_in_table = []
        for content in table_content["result"]:
            if("u_name" in content):
                names_in_table.append(content["u_name"])
        variable["reference_values"] = ", ".join(names_in_table)
        return variable
    
    elif (variable["type"] == 5 or variable["type"] == 18):
        choiceList = []
        for choice in variable["choices"]:
            choiceList.append(choice["value"])
        variable["reference_values"] = ", ".join(choiceList)
        return variable

    elif(variable["value"] != ""):
        variable["displayvalue"] = variable["value"]
        return variable

    else:
        return variable

def fetch_ui_actions(action_field, catalog_variables):
    ui_action_variable_list = []
    for action_variables in action_field:
        action_variable_id = action_variables["name"].split(":")[1] 
        for variable in catalog_variables:
            # print(f"{variable["id"]} = {action_variable_id}")
            if(variable["id"] == action_variable_id):
                ui_action_variable_list.append({"name": variable["name"], "mandatory": action_variables["mandatory"]})
    
    return ui_action_variable_list


def fetch_ui_policy(catalog_item, catalog_variables):
    final_ui_condition_action = []
    for catalog_ui_policy in catalog_item["result"]["ui_policy"]:
        for condition_field in catalog_ui_policy["conditions"]:
            action_field = catalog_ui_policy["actions"]
            ui_action_variable_list = fetch_ui_actions(action_field, catalog_variables)
            variable_id = condition_field["field"].split(":")[1]
            for variable in catalog_variables:
                if variable["id"] == variable_id:
                    ui_condition = {"condition_variable_name":variable["name"],"condition_variable_value": condition_field["value"], "condition_operation": condition_field["oper"], "ui_actions": ui_action_variable_list}
                    final_ui_condition_action.append(ui_condition)
    return final_ui_condition_action


def arrange_api_request(parse_variable_details,variables_template):
    for key in parse_variable_details:
        if key in variables_template:
            variables_template[key] = parse_variable_details[key]
    return variables_template


def get_valuefor_reqfor(variable_info, variables_template, fetched_variables):
        reference = variable_info["reference"]
        sysparm_for_query = "user_name"
        value_for_query = os.environ["SERVICENOW_USERNAME"]
        # print(reference + sysparm_for_query + value_for_query)
        sys_id = get_table_sysid(reference, sysparm_for_query, value_for_query)["result"][0]["sys_id"]
        variables_template[variable_info["name"]] = sys_id
        fetched_variables[variable_info["name"]] = sys_id
        return fetched_variables, variables_template


def get_valuefor_reference(variable_info, variables_template, fetched_variables):
    reference = variable_info["reference"] # u_software

    sample_table_response = get_table_response(reference)
    if "u_name" in sample_table_response["result"][0]:
        sysparm_for_query = "u_name" # name
    elif "name" in sample_table_response["result"][0]:
        sysparm_for_query = "name"

    if(variable_info["name"] in fetched_variables):
        value_for_query = fetched_variables[variable_info["name"]] # python
        sys_id = get_table_sysid(reference, sysparm_for_query, value_for_query)["result"][0]["sys_id"]
        variables_template[variable_info["name"]] = sys_id
        
    return fetched_variables, variables_template



def is_valid_sys_id(value):
    return bool(re.fullmatch(r"[0-9a-fA-F]{32}", value))




def get_valuefor_dynamicvalues(variables_List, variable_info, variables_template, fetched_variables):
    for variables in variables_List:
        if(variables["id"] == variable_info["dynamic_value_field"]):
            reference = variables["reference"]
            sysparm_for_query = "sys_id"# variable_info["dynamic_value_dot_walk_path"]
            value_for_query = variables_template[variables["name"]]
        
            # print(" reference =  "+ reference +" sysparm_for_query =  "+ sysparm_for_query +" value_for_query = "+ value_for_query)
            # print([variable_info["dynamic_value_dot_walk_path"]])
            requireddetails = get_table_sysid(reference, sysparm_for_query, value_for_query)
            if("result" in requireddetails):
                resultValue = requireddetails["result"]
                if(isinstance(resultValue, list)):
                    if(0<len(resultValue)):
                        requireddetails = resultValue[0][variable_info["dynamic_value_dot_walk_path"]]
            finalDetail = ""
            # print(requireddetails)
            if("value" in requireddetails):
                finalDetail = requireddetails["value"]
            else:
                finalDetail = requireddetails
            
            variables_template[variable_info["name"]] = finalDetail
    return fetched_variables, variables_template




def assign_complex_variables(fetched_variables, variables_template, variables_List):
    variables_template = arrange_api_request(fetched_variables, variables_template)
    for variable_info in variables_List:
        # print(variable_info["name"])
        if(variable_info["type"] == 31):            # for requested_for type
            fetched_variables, variables_template = get_valuefor_reqfor(variable_info, variables_template, fetched_variables)

        elif(variable_info["name"] in fetched_variables and variable_info["type"] == 8):
            fetched_variables, variables_template = get_valuefor_reference(variable_info, variables_template, fetched_variables)

        elif (variable_info["dynamic_value_field"] != ""):
            fetched_variables, variables_template = get_valuefor_dynamicvalues(variables_List, variable_info, variables_template, fetched_variables)
    return fetched_variables, variables_template

def fetch_mandatory_variables(variables_template, ui_policy,variables_List):
    missing_mandatory_variables = []
    for variable in variables_List:
        if(variable["mandatory"] == True and variables_template[variable["name"]] == ""):
            missing_mandatory_variables.append(variable["name"])
    for ui_condition in ui_policy:
        if(ui_condition["condition_operation"] == "="):
            if(variables_template[ui_condition["condition_variable_name"]] == ui_condition["condition_variable_value"]):
                ui_actions = ui_condition["ui_actions"]
                for ui_action in ui_actions:
                    if(ui_action["mandatory"] == "true" and variables_template[ui_action["name"]] == ""):
                        missing_mandatory_variables.append(ui_action["name"])
                    
    return variables_template, missing_mandatory_variables

def prepare_api_request(user_query, sys_id, catalog_item_description):
    catalog_item = get_specific_catalog_item(sys_id)
    catalog_variables = catalog_item["result"]["variables"]
    ui_policy = fetch_ui_policy(catalog_item, get_all_variables_List(catalog_variables))
    variables_List = []
    variables_template = {}
    variables_name = []
    for variable in get_all_variables_List(catalog_variables):
        variable = set_reference_value(variable)
        variables_List.append(variable)
        variables_name.append(variable["name"])
        variables_template[variable["name"]] = variable["displayvalue"]
        
    custom_function = function_calling_catVar(variables_List, catalog_item_description, variables_name)
    message = model.predict_messages(
    [HumanMessage(content=f"user query: {user_query}")],
    functions = custom_function
    )

    print(variables_List)
    # print(custom_function)
    # print(json.dumps(message.__dict__))

    if message.additional_kwargs != {}:
        # print("The "additional_kwargs" attribute exists.")
        parse_variable_details = json.loads(message.additional_kwargs["function_call"]["arguments"])
        parse_variable_details,variables_template = assign_complex_variables(parse_variable_details,variables_template, variables_List)
        variables_template, missing_mandatory_variables = fetch_mandatory_variables(variables_template, ui_policy,variables_List)
            
    return parse_variable_details, json.dumps(variables_template), ui_policy, missing_mandatory_variables

In [128]:
prepare_api_request("I need to install python, java", "50572ffac3405a1068d8b132b4013177", catalog_item_description)

{'result': {'short_description': 'Request for Software Services', 'kb_article': '', 'icon': 'images/service_catalog/generic_small.gifx', 'description': '<div>\r\n<ul><li>Please select Business Function/Request Type to display delivery time</li></ul>\r\n</div>', 'availability': 'on_desktop', 'mandatory_attachment': False, 'request_method': '', 'type': 'catalog_item', 'visible_standalone': True, 'sys_class_name': 'sc_cat_item', 'sys_id': '50572ffac3405a1068d8b132b4013177', 'content_type': '', 'order': 0, 'make_item_non_conversational': False, 'owner': '6816f79cc0a8016401c5a33be04be441', 'show_price': False, 'show_quantity': True, 'picture': '', 'url': '', 'catalogs': [{'sys_id': 'e0d08b13c3330100c8b837659bba8fb4', 'active': True, 'title': 'Service Catalog'}], 'name': 'Software Services', 'show_wishlist': True, 'category': {'sys_id': '01d72b3ec3405a1068d8b132b40131ac', 'title': 'Personal Catalogs'}, 'turn_off_nowassist_conversation': False, 'show_delivery_time': True, 'categories': [], 'v

({'business_service': 'Software Installation',
  'software_name': 'Python',
  'add_software': 'true',
  'software_name1': 'Java',
  'requested_for': '6816f79cc0a8016401c5a33be04be441'},
 '{"requested_for": "6816f79cc0a8016401c5a33be04be441", "contact_number": "", "Location": "163c49b037d0200044e0bfc8bcbe5dcc", "project_name": "a581ab703710200044e0bfc8bcbe5de8", "business_service": "effae772c3805a1068d8b132b40131dd", "software_name": "7c29b2afc3441a5068d8b132b4013164", "add_software": "true", "request_type": "Software Installation - Freeware", "duration": "", "software_name1": "3029b2afc3441a5068d8b132b4013165", "add_software1": false, "request_type1": "Software Installation - Freeware", "duration1": "", "software_name2": "", "add_software2": false, "request_type2": "Software Installation - Freeware", "duration2": "", "software_name3": "", "select_your_host_name": ""}',
 [{'condition_variable_name': 'business_service',
   'condition_variable_value': 'effae772c3805a1068d8b132b40131dd',
 

In [129]:
parsed_data = {'business_service': 'Software Installation',
  'software_name': 'Python',
  'add_software': 'true',
  'software_name1': 'Java',
  'requested_for': '6816f79cc0a8016401c5a33be04be441'}

catalog_item = get_specific_catalog_item('50572ffac3405a1068d8b132b4013177')
catalog_variables = catalog_item["result"]["variables"]
ui_policy = fetch_ui_policy(catalog_item, get_all_variables_List(catalog_variables))
variables_List = []
variables_template = {}
variables_name = []
for variable in get_all_variables_List(catalog_variables):
    variable = set_reference_value(variable)
    variables_name.append(variable["name"])
    variables_template[variable["name"]] = variable["displayvalue"]
    for i in parsed_data:
        # print(i)
        if(variable['name'] == i):
            variable['value'] = parsed_data[i]
    variables_List.append(variable)

catalog_item["result"]["variables"] = variables_List
catalog_item["result"]["ui_policy"] = ui_policy

print(catalog_item)
print("")

{'result': {'short_description': 'Request for Software Services', 'kb_article': '', 'icon': 'images/service_catalog/generic_small.gifx', 'description': '<div>\r\n<ul><li>Please select Business Function/Request Type to display delivery time</li></ul>\r\n</div>', 'availability': 'on_desktop', 'mandatory_attachment': False, 'request_method': '', 'type': 'catalog_item', 'visible_standalone': True, 'sys_class_name': 'sc_cat_item', 'sys_id': '50572ffac3405a1068d8b132b4013177', 'content_type': '', 'order': 0, 'make_item_non_conversational': False, 'owner': '6816f79cc0a8016401c5a33be04be441', 'show_price': False, 'show_quantity': True, 'picture': '', 'url': '', 'catalogs': [{'sys_id': 'e0d08b13c3330100c8b837659bba8fb4', 'active': True, 'title': 'Service Catalog'}], 'name': 'Software Services', 'show_wishlist': True, 'category': {'sys_id': '01d72b3ec3405a1068d8b132b40131ac', 'title': 'Personal Catalogs'}, 'turn_off_nowassist_conversation': False, 'show_delivery_time': True, 'categories': [], 'v

In [127]:
catalog_item

{'result': {'short_description': 'Request for Software Services',
  'kb_article': '',
  'icon': 'images/service_catalog/generic_small.gifx',
  'description': '<div>\r\n<ul><li>Please select Business Function/Request Type to display delivery time</li></ul>\r\n</div>',
  'availability': 'on_desktop',
  'mandatory_attachment': False,
  'request_method': '',
  'type': 'catalog_item',
  'visible_standalone': True,
  'sys_class_name': 'sc_cat_item',
  'sys_id': '50572ffac3405a1068d8b132b4013177',
  'content_type': '',
  'order': 0,
  'make_item_non_conversational': False,
  'owner': '6816f79cc0a8016401c5a33be04be441',
  'show_price': False,
  'show_quantity': True,
  'picture': '',
  'url': '',
  'catalogs': [{'sys_id': 'e0d08b13c3330100c8b837659bba8fb4',
    'active': True,
    'title': 'Service Catalog'}],
  'name': 'Software Services',
  'show_wishlist': True,
  'category': {'sys_id': '01d72b3ec3405a1068d8b132b40131ac',
   'title': 'Personal Catalogs'},
  'turn_off_nowassist_conversation'